# Creating function for evaluating  models according to the distribution of error between different values of selected feature 

# Import and watch data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv ('tool touch detection.csv')


In [3]:
df.head()

,SESSION_INFO_ID,INTERVAL_IN_FILE,VALUE,SN,ACCL_X,ACCL_Y,ACCL_Z,BAND_ARR_1,BAND_ARR_2,BAND_ARR_3,...,BATT_LEVEL,DATE,FIRST_PACKET_TS,GW_MAC,GW_NAME,OPERATION_VALID,RSSI,TOOL_MAC,TOOL_NAME,S_I_CREATED_AT
0,6a39aff16c954755887e275fdfbcf82e,7,0,11,32767,-5269,32767,0.240264,0.149872,0.091437,...,67,30:24.0,234,E0D87D23F51F,Machine name,False,-59,C70BA91FEA15,E0000C23F51F,58:54.8
1,6a39aff16c954755887e275fdfbcf82e,20,0,24,32767,-5515,32767,0.173969,0.170729,0.091877,...,67,30:24.0,234,E0D87D23F51F,Machine name,False,-59,C70BA91FEA15,E0000C23F51F,58:54.8
2,6a39aff16c954755887e275fdfbcf82e,36,0,40,32767,-5801,32767,0.264151,0.110667,0.081432,...,67,30:24.0,234,E0D87D23F51F,Machine name,False,-59,C70BA91FEA15,E0000C23F51F,58:54.8
3,6a39aff16c954755887e275fdfbcf82e,45,0,49,32767,-6550,32767,3.655040,2.248858,1.993189,...,67,30:24.0,234,E0D87D23F51F,Machine name,False,-59,C70BA91FEA15,E0000C23F51F,58:54.8
4,6a39aff16c954755887e275fdfbcf82e,52,0,56,32767,-6103,32767,2.797805,1.304610,0.664670,...,67,30:24.0,234,E0D87D23F51F,Machine name,False,-59,C70BA91FEA15,E0000C23F51F,58:54.8


In [4]:
df.shape

(114271, 57)

# Prepare and split data

In [5]:
from sklearn.model_selection import train_test_split


In [6]:
X = df.loc[:, df.columns != 'VALUE']

y = df['VALUE']

from sklearn import preprocessing

number = preprocessing.LabelEncoder()
X['S_I_CREATED_AT'] = number.fit_transform(X.S_I_CREATED_AT)
X['DATE'] = number.fit_transform(X.DATE)
X['GW_MAC'] = number.fit_transform(X.GW_MAC)
X['GW_NAME'] = number.fit_transform(X.GW_NAME)
X['TOOL_MAC'] = number.fit_transform(X.TOOL_MAC)
X['TOOL_NAME'] = number.fit_transform(X.TOOL_NAME)       
X['SESSION_INFO_ID'] = number.fit_transform(X.SESSION_INFO_ID)       



C:\Users\Arie\AppData\Local\Temp/ipykernel_8208/2218193008.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['S_I_CREATED_AT'] = number.fit_transform(X.S_I_CREATED_AT)
C:\Users\Arie\AppData\Local\Temp/ipykernel_8208/2218193008.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['DATE'] = number.fit_transform(X.DATE)
C:\Users\Arie\AppData\Local\Temp/ipykernel_8208/2218193008.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) 


# Train and basically evaluate baseline model

In [8]:
from sklearn.ensemble import RandomForestClassifier


In [9]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [10]:
y_pred = clf.predict(X_test)


In [11]:
from sklearn import metrics


In [12]:
print('F1 Score: ',metrics.f1_score(y_test, y_pred),
     'Accuracy Score: ',metrics.accuracy_score(y_test, y_pred),
     'Precision Score: ',metrics.precision_score(y_test, y_pred),
     'Recall Score: ',metrics.recall_score(y_test, y_pred))
                         

F1 Score:  0.8773367236426991 Accuracy Score:  0.9240126013651478 Precision Score:  0.9802188552188552 Recall Score:  0.793999829540612


# Evaluating the model according to the distribution of error between different values of Session Info Id 

In [13]:
prediction_row_check = pd.DataFrame(columns = ['y pred', 'y'])
prediction_row_check['y']=y_test
prediction_row_check['y pred']=y_pred
prediction_row_check['is prediction wrong'] = (prediction_row_check['y pred'] - prediction_row_check['y'])**2
prediction_row_check['SESSION_INFO_ID']=X_test['SESSION_INFO_ID']
prediction_row_check

,y pred,y,is prediction wrong,SESSION_INFO_ID
22355,1,1,0,111
31832,0,0,0,214
60295,0,0,0,112
68509,0,0,0,88
40746,0,0,0,268
...,...,...,...,...
57562,1,1,0,265
74596,0,0,0,25
42442,0,0,0,252
109119,0,0,0,149


In [14]:
prediction_session_check=pd.DataFrame(prediction_row_check.groupby('SESSION_INFO_ID')['is prediction wrong'].mean())
prediction_session_check

,is prediction wrong
SESSION_INFO_ID,
0,0.000000
3,0.009749
5,0.007463
7,0.000000
10,0.006667
...,...
276,0.015152
277,0.000000
278,0.000000


In [15]:
prediction_session_check=pd.DataFrame(prediction_row_check.groupby('SESSION_INFO_ID')['is prediction wrong'].mean())
prediction_session_check = prediction_session_check.reset_index()
prediction_session_check

,SESSION_INFO_ID,is prediction wrong
0,0,0.000000
1,3,0.009749
2,5,0.007463
3,7,0.000000
4,10,0.006667
...,...,...
207,276,0.015152
208,277,0.000000
209,278,0.000000
210,279,0.000000


In [17]:
import statistics
from statistics import mean

statistics.mean(prediction_row_check['is prediction wrong'])

0.07598739863485211

In [18]:
prediction_session_check['abs difference from average'] = abs(prediction_session_check['is prediction wrong']-statistics.mean(prediction_row_check['is prediction wrong']))
prediction_session_check

,SESSION_INFO_ID,is prediction wrong,abs difference from average
0,0,0.000000,0.075987
1,3,0.009749,0.066238
2,5,0.007463,0.068525
3,7,0.000000,0.075987
4,10,0.006667,0.069321
...,...,...,...
207,276,0.015152,0.060836
208,277,0.000000,0.075987
209,278,0.000000,0.075987
210,279,0.000000,0.075987


In [19]:
print (prediction_session_check['abs difference from average'].mean())


0.07870811504754457


In [20]:
print (prediction_session_check['abs difference from average'].max())


0.87201953377416


In [21]:
prediction_session_check[prediction_session_check['abs difference from average']>0.2]

,SESSION_INFO_ID,is prediction wrong,abs difference from average
40,58,0.313208,0.237220
48,66,0.333333,0.257346
79,102,0.783307,0.707319
120,156,0.948007,0.872020
170,228,0.397004,0.321016


In [22]:
prediction_session_check[prediction_session_check['abs difference from average']>0.2]['SESSION_INFO_ID'].count()

5

In [23]:
prediction_session_check[prediction_session_check['abs difference from average']>0.1]['SESSION_INFO_ID'].count()

10

# Define the whole evaluation process as function

In [24]:
df_for_model_tab = pd.DataFrame(columns = ['validation metric'])
df_for_model_tab['validation metric'] = ('F1', 'Accuracy', 'Precision', 'Recall', 'max difference from error mean', 'mean difference from error mean', 'number of bigger than 0.2 differnece', 'number of bigger than 0.1 differnece')
df_for_model_tab

,validation metric
0,F1
1,Accuracy
2,Precision
3,Recall
4,max difference from error mean
5,mean difference from error mean
6,number of bigger than 0.2 differnece
7,number of bigger than 0.1 differnece


In [25]:
def error_per_session(X, y, model, model_name):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) 

    
    clf = model
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    prediction_row_check = pd.DataFrame(columns = ['y pred', 'y'])
    prediction_row_check['y']=y_test
    prediction_row_check['y pred']=y_pred
    prediction_row_check['SESSION_INFO_ID']=X_test['SESSION_INFO_ID']
    prediction_row_check['is prediction wrong'] = (prediction_row_check['y pred'] - prediction_row_check['y'])**2
    
    prediction_session_check=pd.DataFrame(prediction_row_check.groupby('SESSION_INFO_ID')['is prediction wrong'].mean())
    prediction_session_check = prediction_session_check.reset_index()
    
    prediction_session_check['abs difference from average'] = abs(prediction_session_check['is prediction wrong']-statistics.mean(prediction_row_check['is prediction wrong']))
    
    f1 = metrics.f1_score(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    max_difference_from_error_mean = prediction_session_check['abs difference from average'].max()
    mean_difference_from_error_mean = prediction_session_check['abs difference from average'].mean()
    number_of_bigger_than_02_differnece = prediction_session_check[prediction_session_check['abs difference from average']>0.2]['SESSION_INFO_ID'].count()
    number_of_bigger_than_01_differnece = prediction_session_check[prediction_session_check['abs difference from average']>0.1]['SESSION_INFO_ID'].count()
    df_for_model_tab[model_name] = f1, accuracy, precision, recall, max_difference_from_error_mean , mean_difference_from_error_mean, number_of_bigger_than_02_differnece, number_of_bigger_than_01_differnece



# Train and evaluate models with the new function

In [26]:
error_per_session(X, y, RandomForestClassifier(max_depth=2, random_state=0), 'Random Forest')
df_for_model_tab

,validation metric,Random Forest
0,F1,0.876719
1,Accuracy,0.923896
2,Precision,0.979000
3,Recall,0.793788
4,max difference from error mean,0.874945
5,mean difference from error mean,0.079615
6,number of bigger than 0.2 differnece,5.000000
7,number of bigger than 0.1 differnece,10.000000


In [27]:
from sklearn.tree import DecisionTreeClassifier


In [28]:
error_per_session(X, y, DecisionTreeClassifier(), 'Decision Tree')
df_for_model_tab

,validation metric,Random Forest,Decision Tree
0,F1,0.876719,0.989487
1,Accuracy,0.923896,0.992853
2,Precision,0.979000,0.988342
3,Recall,0.793788,0.990635
4,max difference from error mean,0.874945,0.072720
5,mean difference from error mean,0.079615,0.007434
6,number of bigger than 0.2 differnece,5.000000,0.000000
7,number of bigger than 0.1 differnece,10.000000,0.000000


In [29]:
from sklearn.ensemble import AdaBoostClassifier

In [30]:
error_per_session(X, y, AdaBoostClassifier(), 'Ada Boost')
df_for_model_tab

,validation metric,Random Forest,Decision Tree,Ada Boost
0,F1,0.876719,0.989487,0.976275
1,Accuracy,0.923896,0.992853,0.984015
2,Precision,0.979000,0.988342,0.979243
3,Recall,0.793788,0.990635,0.973325
4,max difference from error mean,0.874945,0.072720,0.105227
5,mean difference from error mean,0.079615,0.007434,0.015124
6,number of bigger than 0.2 differnece,5.000000,0.000000,0.000000
7,number of bigger than 0.1 differnece,10.000000,0.000000,1.000000
